In [6]:
import json
from json_repair import repair_json

In [ ]:
sample_message = """{"message_id": 31329, "text": "expande tu imagen ! ! ! ! ! ! ! ! ! si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar", "label": 1, "label_text": "spam", "subject": "expande tu imagen ! ! ! ! ! ! ! ! !", "message": "si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar\n", "date": "2005-01-19"}"""
sample_messaage = repair_json(sample_message)
sample_message_json = json.loads(sample_messaage)
formatted_message_json = json.dumps(sample_message_json, indent=2)
print(formatted_message_json)

{
  "message_id": 31329,
  "text": "expande tu imagen ! ! ! ! ! ! ! ! ! si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar",
  "label": 1,
  "label_text": "spam",
  "subject": "expande tu imagen ! ! ! ! ! ! ! ! !",
  "message": "si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repb

In [9]:
# Count elements in test.jsonl
test_list = []
with open('test.jsonl', 'r') as f:
    for line in f:
        test_item = json.loads(repair_json(line))
        test_list.append(test_item)


# Count elements in train.jsonl
train_list = []
with open('train.jsonl', 'r') as f:
    for line in f:
        train_item = json.loads(repair_json(line))
        train_list.append(train_item)

print(f"Number of elements in test.jsonl: {len(test_list)}")
print(f"Number of elements in train.jsonl: {len(train_list)}")

Number of elements in test.jsonl: 2000
Number of elements in train.jsonl: 31716
